In [1]:
from sendgrid.helpers.mail import Mail, Email, To, Content
from agents.model_settings import ModelSettings
from pydantic import BaseModel, Field
from agents import Agent, WebSearchTool, trace, Runner, function_tool
from dotenv import load_dotenv
import asyncio
import sendgrid
import os
from typing import Dict
from IPython.display import display, Markdown

In [2]:
load_dotenv(override=True)

True

In [5]:
import logfire
logfire.configure(
    service_name="sale email manager",
    send_to_logfire=False
)
logfire.instrument_openai_agents()

In [6]:
from langfuse import get_client
langfuse = get_client()
langfuse.auth_check()


True

In [7]:
INSTRUCTIONS = """
You are a research assistant specializing in rapid information retrieval. Given a user's search term, your task is to perform a web search and create a concise summary of the findings. The summary must be 2-3 paragraphs, staying under 300 words.
Focus on capturing the main points and critical details. Your writing should be succinct and to the point; complete sentences and formal grammar are not necessary. This output will be used to synthesize a larger report, so it's crucial to capture the essence of the search results and eliminate all filler.
Your final response should only contain the summary itself—no additional commentary, introductions, or conversational text.
You have given a tool to perform web searches. Use it to gather information relevant to the user's query.
"""

search_agent = Agent(
    name="Search agent",
    instructions=INSTRUCTIONS,
    tools=[WebSearchTool(search_context_size="low")],
    model="gpt-4o-mini",
    model_settings=ModelSettings(tool_choice="required"),
)

In [8]:
message = "Latest AI Agent frameworks in 2025"

with trace("Search"):
    result = await Runner.run(search_agent, message)

display(Markdown(result.final_output))

18:03:31.538 OpenAI Agents trace: Search
18:03:31.543   Agent run: 'Search agent'
18:03:31.545     Responses API with 'gpt-4o-mini'


In 2025, several AI agent frameworks have emerged, each offering unique capabilities for developing intelligent, autonomous systems.

**Agent Lightning** is a flexible framework that enables reinforcement learning-based training of large language models (LLMs) for any AI agent. It decouples agent execution from training, allowing seamless integration with existing agents developed through various methods, including frameworks like LangChain and OpenAI Agents SDK. By formulating agent execution as a Markov decision process, Agent Lightning introduces a hierarchical RL algorithm, LightningRL, facilitating complex interaction logic handling. ([arxiv.org](https://arxiv.org/abs/2508.03680?utm_source=openai))

**Polymorphic Combinatorial Framework (PCF)** leverages LLMs and mathematical frameworks to guide the design of adaptive AI agents. It enables real-time parameter reconfiguration through combinatorial spaces, allowing agents to dynamically adapt their behavioral traits. Grounded in combinatorial logic and rough fuzzy set theory, PCF defines a multidimensional parameter space (SPARK) to capture agent behaviors, supporting scalable, dynamic, and ethical AI applications. ([arxiv.org](https://arxiv.org/abs/2508.01581?utm_source=openai))

**Cognitive Kernel-Pro** is an open-source, multi-module agent framework designed to democratize the development and evaluation of advanced AI agents. It systematically investigates the curation of high-quality training data for Agent Foundation Models across domains like web, file, code, and general reasoning. The framework introduces novel strategies for agent test-time reflection and voting to enhance robustness and performance, achieving state-of-the-art results among open-source agents. ([arxiv.org](https://arxiv.org/abs/2508.00414?utm_source=openai))

**AutoAgent** is a fully-automated, zero-code framework that enables users to create and deploy LLM agents through natural language alone. Operating as an autonomous Agent Operating System, AutoAgent comprises components like Agentic System Utilities, LLM-powered Actionable Engine, Self-Managing File System, and Self-Play Agent Customization module. It serves as a versatile multi-agent system for general AI assistants, demonstrating effectiveness in generalist multi-agent tasks and surpassing existing state-of-the-art methods. ([arxiv.org](https://arxiv.org/abs/2502.05957?utm_source=openai))

**Microsoft's Semantic Kernel** is an open-source framework that integrates AI models, such as LLMs, into applications. It supports multiple programming languages (e.g., Python, Java, C#) and offers task orchestration and real-time analytics, making it suitable for enhancing legacy systems with AI capabilities. ([aaditech.com](https://aaditech.com/2025/01/08/ai-agent-frameworks-for-2025/?utm_source=openai))

**Amazon's Bedrock AgentCore** is a platform designed to simplify the development and deployment of advanced AI agents. It includes modular services supporting the full production lifecycle, such as AgentCore Runtime for scalable serverless deployment, AgentCore Memory for context management, and AgentCore Identity for secure service access. The platform emphasizes flexibility, allowing developers to use only the components they need, facilitating scalable and secure AI agent development. ([techradar.com](https://www.techradar.com/pro/aws-looks-to-super-charge-ai-agents-with-amazon-bedrock-agentcore?utm_source=openai))

These frameworks represent significant advancements in AI agent development, offering diverse tools and methodologies to create intelligent, adaptable, and secure AI systems. 

In [9]:
HOW_MANY_SEARCHES = 3

INSTRUCTIONS = f"You are a helpful research assistant. Given a query, come up with a set of web searches \
to perform to best answer the user query. Output {HOW_MANY_SEARCHES} terms to query for."


class WebSearchItem(BaseModel):
    reason: str = Field(description="Your reasoning for why this search is important to the query.")

    query: str = Field(description="The search term to use for the web search.")


class WebSearchPlan(BaseModel):
    searches: list[WebSearchItem] = Field(description="A list of web searches to perform to best answer the query.")


planner_agent = Agent(
    name="PlannerAgent",
    instructions=INSTRUCTIONS,
    model="gpt-4o-mini",
    output_type=WebSearchPlan,
)

In [10]:

message = "Latest AI Agent frameworks in 2025"

with trace("Search"):
    result = await Runner.run(planner_agent, message)
    print(result.final_output)

18:06:54.650 OpenAI Agents trace: Search
18:06:54.652   Agent run: 'PlannerAgent'
18:06:54.656     Responses API with 'gpt-4o-mini'
searches=[WebSearchItem(reason='To find the most current information on AI agent frameworks available in 2025.', query='latest AI agent frameworks 2025'), WebSearchItem(reason='To explore industry reports or articles that highlight emerging AI technologies and frameworks.', query='AI agent technologies trends 2025'), WebSearchItem(reason='To identify leading companies and their contributions to AI agent frameworks in 2025.', query='top companies AI agent frameworks 2025')]


In [11]:
sender_email = "bhavin@advant.xyz"
receiver_email = 'bhavin2004k@gmail.com'

In [12]:
@function_tool
def send_html_email(subject: str, html_body: str) -> Dict[str, str]:
    """ Send out an email with the given subject and HTML body to all sales prospects """
    sg = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))
    from_email = Email(sender_email)
    to_email = To(receiver_email) 
    content = Content("text/html", html_body)
    mail = Mail(from_email, to_email, subject, content).get()
    response = sg.client.mail.send.post(request_body=mail)
    return {"status": "success"}
send_html_email

FunctionTool(name='send_html_email', description='Send out an email with the given subject and HTML body to all sales prospects', params_json_schema={'properties': {'subject': {'title': 'Subject', 'type': 'string'}, 'html_body': {'title': 'Html Body', 'type': 'string'}}, 'required': ['subject', 'html_body'], 'title': 'send_html_email_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x7a16d82007c0>, strict_json_schema=True, is_enabled=True)

In [13]:
INSTRUCTIONS = """You are able to send a nicely formatted HTML email based on a detailed report.
You will be provided with a detailed report. You should use your tool to send one email, providing the 
report converted into clean, well presented HTML with an appropriate subject line."""

email_agent = Agent(
    name="Email agent",
    instructions=INSTRUCTIONS,
    tools=[send_html_email],
    model="gpt-4o-mini",
)



In [14]:
INSTRUCTIONS = """You are a senior researcher preparing a detailed report based on a research query and supporting materials from an assistant. 
First, create a clear outline describing the report’s structure and sections. Then, write the full report in Markdown format, ensuring it is thorough,
well-organized, and at least 1000 words long (about 5–10 pages). Include an introduction, detailed analysis, and a conclusion, and ensure the flow is 
logical and cohesive."""


class ReportData(BaseModel):
    short_summary: str = Field(description="A short 2-3 sentence summary of the findings.")

    markdown_report: str = Field(description="The final report")

    follow_up_questions: list[str] = Field(description="Suggested topics to research further")


writer_agent = Agent(
    name="WriterAgent",
    instructions=INSTRUCTIONS,
    model="gpt-4o-mini",
    output_type=ReportData,
)

In [15]:
async def plan_searches(query: str):
    """ Use the planner_agent to plan which searches to run for the query """
    print("Planning searches...")
    result = await Runner.run(planner_agent, f"Query: {query}")
    print(f"Will perform {len(result.final_output.searches)} searches")
    return result.final_output

async def perform_searches(search_plan: WebSearchPlan):
    """ Call search() for each item in the search plan """
    print("Searching...")
    tasks = [asyncio.create_task(search(item)) for item in search_plan.searches]
    results = await asyncio.gather(*tasks)
    print("Finished searching")
    return results

async def search(item: WebSearchItem):
    """ Use the search agent to run a web search for each item in the search plan """
    input = f"Search term: {item.query}\nReason for searching: {item.reason}"
    result = await Runner.run(search_agent, input)
    return result.final_output

### The next 2 functions write a report and email it

In [16]:
async def write_report(query: str, search_results: list[str]):
    """ Use the writer agent to write a report based on the search results"""
    print("Thinking about report...")
    input = f"Original query: {query}\nSummarized search results: {search_results}"
    result = await Runner.run(writer_agent, input)
    print("Finished writing report")
    return result.final_output

async def send_email(report: ReportData):
    """ Use the email agent to send an email with the report """
    print("Writing email...")
    result = await Runner.run(email_agent, report.markdown_report)
    print("Email sent")
    return report

### Showtime!

In [17]:
query ="Best and Latest Web Series From Marvel"

with trace("Research trace"):
    print("Starting research...")
    search_plan = await plan_searches(query)
    search_results = await perform_searches(search_plan)
    report = await write_report(query, search_results)
    await send_email(report)  
    print("Hooray!")




18:15:17.380 OpenAI Agents trace: Research trace
Starting research...
Planning searches...
18:15:17.382   Agent run: 'PlannerAgent'
18:15:17.385     Responses API with 'gpt-4o-mini'
Will perform 3 searches
Searching...
18:15:20.171   Agent run: 'Search agent'
18:15:20.171   Agent run: 'Search agent'
18:15:20.171   Agent run: 'Search agent'
               Agent run: 'Search agent'
18:15:20.172     Responses API with 'gpt-4o-mini'
               Agent run: 'Search agent'
18:15:20.174     Responses API with 'gpt-4o-mini'
               Agent run: 'Search agent'
18:15:20.175     Responses API with 'gpt-4o-mini'
Finished searching
Thinking about report...
18:15:24.943   Agent run: 'WriterAgent'
18:15:24.944     Responses API with 'gpt-4o-mini'
Finished writing report
Writing email...
18:16:15.486   Agent run: 'Email agent'
18:16:15.486     Responses API with 'gpt-4o-mini'
18:17:04.694     Function: send_html_email
18:17:06.666     Responses API with 'gpt-4o-mini'
Email sent
Hooray!
